# Minflux Data Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.lines import Line2D   
import pandas as pd
import ipywidgets as widgets

In [ ]:
data = np.load() # Insert your diectory here

In [ ]:
#data.dtype
#Unique track ids
#np.unique(data['tid'])

itr = iterations
tic = 
loc = localization (corrected based on beamline monitoring)
lnc = (when beam monitoring used) non-corrected localizations 
eco = effective counts at offset
ecc = 
efo = effective frequency at offset 
efc =
sta =
cfr = center frequency ratio (efc/ efo)
dcr = detection channel ratio
ext = 
gvy = 
gvx = 
eoy = 
eox = 
dmz = 
lcy = 
lcx = 
lcz = 
fbg = 
sqi = 
gri = 
tim = time relative to the start
tid = 
vld = 
act = 
dos = 
ssy = 
fbg = fluorescence background level

In [ ]:
def msd_fit(t, D):
        return 4 * D * t
    
data_list = []
for i, tracks in enumerate(np.unique(data['tid'])):
    #define model to fit MSD
    
    
    track = data[data['tid'] == tracks]["itr"]["loc"]
    #Flatten the array to make it easier to work with
    data_flat = track.reshape(-1, track.shape[-1])
    
    #Remove rows with nan values
    valid_data = data_flat[~np.isnan(data_flat).any(axis=1)]
    #Extracting x and y values
    x_values = (valid_data[3:, 0] * 1e6) 
    y_values = valid_data[3:, 1] * 1e6
    timepoints = data[data['tid'] == tracks]["tim"]
    if len(data[data['tid'] == tracks]["tim"]) >1:
        #Extracting diffusion coefficients
        delta_x = np.diff(x_values) 
        delta_y = np.diff(y_values)
        squared_disp = delta_x**2 + delta_y**2
        lag_times = np.diff(data[data['tid'] == tracks]["tim"])
        msd = np.mean(squared_disp)
        popt, _ = curve_fit(msd_fit, lag_times, squared_disp)
        fitted_msd = msd_fit(lag_times, *popt)
        D = abs(popt[0])
        track_duration = max(data[data['tid'] == tracks]["tim"])-min(data[data['tid'] == tracks]["tim"])
        localization_count = len(x_values)
        

        data_list.append({
                "Track_ID": tracks,
                "DiffCoeff": D,
                "X": x_values,
                "Y": y_values,
                "Track_Duration":track_duration,
                "Localization_Count":localization_count,
                "Timepoints": timepoints,
                
            })
        
df = pd.DataFrame(data_list)


min_x = np.abs(min(df["X"].apply(lambda x: min(x))))
max_x = np.abs(max(df["X"].apply(lambda x: max(x)))) 
min_y = np.abs(min(df["Y"].apply(lambda x: min(x))))
max_y = np.abs(max(df["Y"].apply(lambda x: max(x)))) 

if min_x < max_x:
    new_max_x = max_x
    new_min_x = min_x
elif min_x > max_x:
    new_max_x = min_x
    new_min_x = max_x
if min_y < max_y:
    new_max_y = max_y
    new_min_y = min_y
elif min_y > max_y:
    new_max_y = min_y
    new_min_y = max_y

# Interactive Figure

In [ ]:
%matplotlib widget

class PointBrowser:
    def __init__(self, fig, ax, ax2, df):
        self.fig = fig
        self.ax = ax
        self.ax2 = ax2
        self.df = df
        self.lastind = None
        self.lines = []
        self.selected_line = None  # Track the currently selected line object
        self.original_line_props = {}  # Store original line properties
        self.text = ax.text(0.05, 0.95, 'Selected: none', transform=ax.transAxes, va='top')
        
    def plot_initial_lines(self):
        # Plot all lines with a unique identifier
        for index, row in self.df.iterrows():
            #definine max and min to properly plot axis
            corrected_x = new_max_x - np.abs(row["X"])
            corrected_y = new_max_y - np.abs(row["Y"])
            
            line, = self.ax.plot(corrected_x, corrected_y, ms=12, picker=1, label=str(row["Track_ID"]), alpha = 0.7)
            self.ax.set_xlabel(u'X / $\mathrm{\mu m}$', fontsize=20)
            self.ax.set_ylabel(u'Y / $\mathrm{\mu m}$', fontsize=20)
            self.ax.tick_params(labelsize=20)
            self.ax.axis("equal")
            self.lines.append((line, index))  # Store line and corresponding DataFrame index
            self.original_line_props[line] = {'linewidth': line.get_linewidth(), 'color': line.get_color()}  # Store original properties
            
    def on_pick(self, event):
        # Find which line was picked
        for line, index in self.lines:
            if line == event.artist:
                if self.selected_line is not None:
                    # Revert the previously selected line to its original state
                    orig_props = self.original_line_props[self.selected_line]
                    self.selected_line.set_linewidth(orig_props['linewidth'])
                    self.selected_line.set_color(orig_props['color'])
                
                self.lastind = index
                self.selected_line = line  # Update the currently selected line
                self.update()
                return

    def update(self):
        if self.lastind is None:
            return
        
        dataind = self.lastind
        track_id = self.df.loc[dataind, "Track_ID"]
        diff_coeff = self.df.loc[dataind, "DiffCoeff"]
        duration = self.df.loc[dataind, "Track_Duration"]
        count = self.df.loc[dataind, "Localization_Count"]
        time =   self.df.loc[dataind, "Timepoints"]

        # Highlight the selected line
        self.selected_line.set_linewidth(4)  # Make line wider
        self.selected_line.set_color('red')  # Change line color to red
        
        self.ax.figure.canvas.draw_idle()  # Only redraw the necessary parts
        
        
        
        selected_x_list = self.df.loc[dataind, "X"]
        selected_y_list = self.df.loc[dataind, "Y"]
        #definine max and min to properly plot axis
        min_x = np.abs(min(selected_x_list))
        max_x = np.abs(max(selected_x_list))
        min_y = np.abs(min(selected_y_list))
        max_y = np.abs(max(selected_y_list))
        if min_x < max_x:
            new_max_x = max_x
            new_min_x = min_x
        elif min_x > max_x:
            new_max_x = min_x
            new_min_x = max_x
        if min_y < max_y:
            new_max_y = max_y
            new_min_y = min_y
        elif min_y > max_y:
            new_max_y = min_y
            new_min_y = max_y
        
        nm_correction = 1000
        corrected_x =  (new_max_x - np.abs(selected_x_list)) * nm_correction
        corrected_y = (new_max_y - np.abs(selected_y_list)) * nm_correction
        
        max_axis = max(max(corrected_x), max(corrected_y))
        
        self.ax2.clear()
        self.ax2.plot(corrected_x, corrected_y,markersize=4, alpha = 1, linewidth = 0.2)
        #self.ax2.set_aspect('equal', adjustable='datalim')
        self.ax2.set_title(f"Track ID: {track_id}", fontsize = 30)
        self.ax2.set_xlabel(u'X / nm', fontsize = 20)
        self.ax2.set_ylabel(u'Y / nm', fontsize = 20)
        self.ax2.set_xlim(-5,max_axis+5)
        self.ax2.set_ylim(-5,max_axis+5)
        self.ax2.tick_params(labelsize = 20)
        self.ax2.text(0, -0.15, f'$D$ = {diff_coeff:1.3f} $\mathrm{{\mu m^2 s^{{-1}}}}$ \n$\Delta t$ = {duration:1.2f} s \n$t_\mathrm{{Begin}}$: {min(time):1.2f} s\n$t_\mathrm{{End}}$: {max(time):1.2f} s\nLocalizations = {count}',  transform=ax2.transAxes, va = "top", fontsize = 20, bbox=dict( facecolor='white', alpha=0.5))
        self.text.set_text(f'Selected: {track_id}')
        self.fig.canvas.draw()
        
    def search_and_highlight(self, track_id_search):
        
        if track_id_search.strip() == '':
            print("Please enter a valid Track ID.")
            return

        try:
            track_id_search = int(track_id_search)
        except ValueError:
            print("Track ID must be an integer.")
            return

        #Reset the previously selected line to its original state
        if self.selected_line is not None:
            orig_props = self.original_line_props[self.selected_line]
            self.selected_line.set_linewidth(orig_props['linewidth'])
            self.selected_line.set_color(orig_props['color'])
            self.selected_line = None

        found = False
        for line, index in self.lines:
            if self.df.loc[index, "Track_ID"] == track_id_search:
                self.lastind = index
                self.selected_line = line
                self.update()
                found = True
                break

        if not found:
            self.text.set_text(f'Selected: Track {track_id_search} not found')
        else:
            self.text.set_text(f'Selected: {track_id_search}')
            
            
#Setup for interactive plotting
fig, (ax, ax2) = plt.subplots(2, 1, figsize=(10, 20))
fig.subplots_adjust(bottom=0.2)

#plt.axis("equal")

browser = PointBrowser(fig, ax, ax2, df)
browser.plot_initial_lines()
fig.canvas.mpl_connect('pick_event', browser.on_pick)
plt.tick_params(labelsize=20)
#function to be called when search input is submitted
def on_search_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        browser.search_and_highlight(change['new'])

search_bar = widgets.Text(
    value='',
    placeholder='Enter Track ID',
    description='Search:',
    disabled=False
)

search_bar.observe(on_search_change, 'value')

display(search_bar)
plt.show()